In [1]:
# import needed libs
import cv2
import math
import numpy as np
import mediapipe as mp
import pandas as pd
from mio import ResultIO
from paths import *
from google.protobuf.json_format import MessageToDict
import json

## Define constants

In [2]:
# CONFIG
VID_SUF = ".mp4"
S = "/"

# relinquit
DESIRED_HEIGHT = 480
DESIRED_WIDTH = 480

# common frame size in the videos
FRAME_WIDTH = 1280
FRAME_HEIGHT = 720

## define mediapipe vars

In [3]:
# prepare mp
mp_hands = mp.solutions.hands
# Import drawing_utils and drawing_styles.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

In [4]:
def lm_has_side_and_is_at(lm, side): 
    # side: 0 == left, 1 == right
    if lm.multi_handedness is None: 
        return False, 0
    elif len(lm.multi_handedness) == 1: 
        handedness_dict = MessageToDict(lm.multi_handedness[0])
        return handedness_dict["classification"][0]["index"] == side, 0
    elif len(lm.multi_handedness) == 2:
        return True, side
    else: 
        return False, 0

In [5]:
el = list(mp_hands.HAND_CONNECTIONS)
el = [list(element) for element in el]

In [11]:
def sol2json(d, json_path): 
    with open(json_path, 'w') as fl:
        has, at = lm_has_side_and_is_at(d, 0)
        if has: 
            ml = (MessageToDict(d.multi_hand_landmarks[at])["landmark"]) # 0 is one of the hands, do this first
            my_dict = {str(i): (d['x'], d['y'], d['z']) for i, d in enumerate(ml)}
        else: 
            my_dict = {str(i): (0, 0, 0) for i in range(21)} # default (0, 0, 0) for all nodes
        outdict = {"edges": el, "features": my_dict}
        fl.write(json.dumps(outdict, separators=(',', ':')))

In [12]:
for vd in os.listdir(det_dir): 
    for clip in os.listdir(det_dir + vd + "/"): 
        vid_graph_dir = graph_dir + vd + "/" + clip + "/"
        mk(vid_graph_dir)   # ?
        print(clip)
        for g in os.listdir(det_dir + vd + "/" + clip + "/"): 
            clip_dir = det_dir + vd + "/" + clip + "/"
            if g.endswith(".pkl"): 
                name, ext = g.split(".")
                d = ResultIO.read(os.path.join(clip_dir, g))
                json_path = os.path.join(graph_dir, "{}.json".format(name))
                sol2json(d, json_path)

yeah
